### Задача - посчитать среднее время жизни пользователей, которые выставили более 100 оценок

Используем файл с оценками фильмов ml-latest-small/ratings.csv.  
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
# Исходный датафрейм
df3 = pd.read_csv('ml-latest-small/ratings.csv')
df3.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


#### Решение 1.

In [3]:
df4 = df3.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
df4.head()

timestamp                  
               min         max count
userId                              
1       1260759108  1260759205    20
2        835355395   835356246    76
3       1298861589  1298932787    51
4        949778714   949982274   204
5       1163373044  1163375145   100

In [4]:
# Оставляем (фильтруем) в датафрейме только пользователей, которые поставили более 100 оценок
df5 = df4.loc[df4['timestamp']['count'] > 100]['timestamp'].sort_values('count')
df5.head()

,min,max,count
userId,,,
623,1225255417,1225258519,103
257,1338006929,1449941803,103
67,854711770,854715906,103
36,847056510,853005800,104
217,1108160000,1108160981,104


In [5]:
df5['lifetime'] = df5['max'] - df5['min']
df5.sort_values('lifetime').head()

,min,max,count,lifetime
userId,,,,
536,829471719,829471925,109,206
609,1029869519,1029870367,140,848
316,1460822646,1460823562,140,916
217,1108160000,1108160981,104,981
110,840100360,840101508,120,1148


In [6]:
df5['lifetime'] = df5['lifetime'].div(31536000)
df5.head()

,min,max,count,lifetime
userId,,,,
623,1225255417,1225258519,103,0.000098
257,1338006929,1449941803,103,3.549432
67,854711770,854715906,103,0.000131
36,847056510,853005800,104,0.188651
217,1108160000,1108160981,104,0.000031


In [34]:
## Находим среднее время жизни пользователей, которые поставили более 100 оценок, в годах
df5.lifetime.mean()

1.2709445538309359

####  Решение 2.

In [35]:
# Находим максимальную дату для каждого пользователя
df_max = df3.groupby('userId').max()[['timestamp']]
df_max.rename(columns = {'timestamp':'timestamp_max'}, inplace = True)
df_max.head()

,timestamp_max
userId,
1,1260759205
2,835356246
3,1298932787
4,949982274
5,1163375145


In [36]:
# Находим минимальную дату для каждого пользователя
df_min = df3.groupby('userId').min()[['timestamp']]
df_min.rename(columns = {'timestamp':'timestamp_min'}, inplace = True)
df_min.head()

,timestamp_min
userId,
1,1260759108
2,835355395
3,1298861589
4,949778714
5,1163373044


In [37]:
# Объединяем максимальную и минимальную даты для каждого пользователя
df_new = df_min.join(df_max)
df_new.head()

,timestamp_min,timestamp_max
userId,,
1,1260759108,1260759205
2,835355395,835356246
3,1298861589,1298932787
4,949778714,949982274
5,1163373044,1163375145


In [38]:
# Находим время жизни для каждого пользователя
df_new['lifetime'] = df_new['timestamp_max'] - df_new['timestamp_min']
df_new.head()

,timestamp_min,timestamp_max,lifetime
userId,,,
1,1260759108,1260759205,97
2,835355395,835356246,851
3,1298861589,1298932787,71198
4,949778714,949982274,203560
5,1163373044,1163375145,2101


In [39]:
# Считаем количество оценок для каждого пользователя
df4 = df3.groupby('userId').count()
df4.head()

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [40]:
# Оставляем (фильтруем) в датафрейме только пользователей, которые поставили более 100 оценок
df5 = df4.loc[df4['rating'] > 100].sort_values('rating')
df5.head()

,movieId,rating,timestamp
userId,,,
623,103,103,103
257,103,103,103
67,103,103,103
36,104,104,104
217,104,104,104


In [41]:
# Соединяем данные пользователей с их временем жизни
joined = df5.merge(df_new, how = 'left', on = 'userId')
joined.head()

,movieId,rating,timestamp,timestamp_min,timestamp_max,lifetime
userId,,,,,,
623,103,103,103,1225255417,1225258519,3102
257,103,103,103,1338006929,1449941803,111934874
67,103,103,103,854711770,854715906,4136
36,104,104,104,847056510,853005800,5949290
217,104,104,104,1108160000,1108160981,981


In [28]:
# Переводим секунды в годы
joined['lifetime'] = joined['lifetime'].div(31536000)

In [29]:
# Находим среднее время жизни пользователей, которые поставили более 100 оценок, в годах
joined.lifetime.mean()

1.2709445538309359